In [ ]:
'''
    Creating DataFrames from existing semi-structured data. And working with these DataFrames.
    
    Task-1: Create a DataFrame "nytDF1" by reading the two files:
            "/scratch/ISE495/data/nyt_part1_1.json" and "/scratch/ISE495/data/nyt_part1_2.json"
    Task-2: Create another DataFrame "nytDF2" by reading the two files:
            "/scratch/ISE495/data/nyt_part2_1.json" and "/scratch/ISE495/data/nyt_part2_2.json"
    Task-3: Get acquainted with the schema of these two DataFrames.
    Task-4: Find the number of books whose current rank is better than the last week's rank.
    Task-5: Print the number of books per number of weeks on the best seller's list.
            Print it in descending order for the number of weeks.
            E.g., 100 books with '10' weeks on the list, 200 books with '8' week on the list etc.
    Task-5: Find the number of books with price <= $25 and that are exactly one week on the best seller's list.
    Task-6: Does price of a book have an affect on the book being on the best seller's list?
    Task-7: How many unique authors are considered in the data?
    Task-8: Is there any correlation between the authors and the price of their books?
    Task-9: Are there any authors who are very well-known for being on the best seller's list?
    Task-10: Consider top-4 authors based on the number of books per author in the given data.
            For these authors, find number of books per number of weeks on the best seller's list
            only for the books for which the price is between $15 and $22.50.
'''

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import array_contains


In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
print port,     os.getcwd().split("/")[2]

config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
            .set('spark.cores.max', '2')\
            .set('spark.port.maxRetries','200')\
            .set('spark.ui.port', port)\
            .set('spark.master', '')\
            .set('spark.app.name', "structuredComputation")
print config.getAll()

#sc = pyspark.SparkContext(appName="GettingStarted", master='spark://polyp1:7077',conf = config)

4214 rus217
[('spark.ui.port', u'4214'), ('spark.port.maxRetries', u'200'), ('spark.cores.max', u'2'), ('spark.master', u'spark://polyp1:7077'), ('spark.executor.memory', u'512M'), ('spark.app.name', u'structuredComputation')]


In [3]:
ss = pyspark.sql.SparkSession.builder.config(conf = config).getOrCreate()

print ss.conf.get('spark.app.name')
print ss.conf.get('spark.master')

structuredComputation
spark://polyp1:7077


In [4]:
#### Input data
sc = ss.sparkContext
nytDF1_1 = ss.read.format('json').\
            load(["/scratch/ISE495/data/nyt_part1_1.json","/scratch/ISE495/data/nyt_part1_2.json"])
nytDF2_1 = ss.read.format('json').\
            load(["/scratch/ISE495/data/nyt_part2_1.json","/scratch/ISE495/data/nyt_part2_2.json"])

In [5]:
nytDF1_1.take(1)

[Row(_id=Row($oid=u'5b4aa4ead3089013507db18b'), amazon_product_url=u'http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author=u'Dean R Koontz', bestsellers_date=Row($date=Row($numberLong=u'1211587200000')), description=u'Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', weeks_on_list=Row($numberInt=u'1'))]

In [6]:
nytDF2_1.take(1)

[Row(_id=Row($oid=u'5b4aa4ead3089013507db18b'), price=Row($numberDouble=None, $numberInt=u'27'), published_date=Row($date=Row($numberLong=u'1212883200000')), publisher=u'Bantam', rank=Row($numberInt=u'1'), rank_last_week=Row($numberInt=u'0'), title=u'ODD HOURS')]

   # Task-3: Get acquainted with the schema of these two DataFrames.
    

In [7]:
print 'The schema of nyt DF1 is: \n\n',nytDF1_1.printSchema()

The schema of nyt DF1 is: 

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- amazon_product_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- bestsellers_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- weeks_on_list: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)

None


In [8]:
print 'The schema of nyt DF2 is: \n\n',nytDF2_1.printSchema()

The schema of nyt DF2 is: 

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- published_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rank: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- rank_last_week: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- title: string (nullable = true)

None


In [54]:
def new(x):
    return (x[0][0],float(max(x[1][0],x[1][1])),x[2][0][0],x[3],x[4][0],x[5][0],x[6])


In [55]:
nyt2_new = nytDF2_1.rdd.map(new)

In [56]:
nyt2_2 = nyt2_new.toDF(['id','price','published_date','publisher','rank','rank_last_week','title'])

In [57]:
nytDF1_1.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|          [3]|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587200000]]|A woman's happy m...|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|Patricia Cornwell|[[1211587200000]]|A Massachusetts s...|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|  Chuck Palahniuk|[[1211587200000]]|An aging porn que...|          [1]|
+--------------------+--------------------+-----------------+-----------------+-

In [58]:
nytDF2_1.show(5)

+--------------------+------------+-----------------+-------------+----+--------------+--------------------+
|                 _id|       price|   published_date|    publisher|rank|rank_last_week|               title|
+--------------------+------------+-----------------+-------------+----+--------------+--------------------+
|[5b4aa4ead3089013...|   [null,27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|
|[5b4aa4ead3089013...|[25.99,null]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|
|[5b4aa4ead3089013...|[24.95,null]|[[1212883200000]]| St. Martin's| [3]|           [2]|LOVE THE ONE YOU'...|
|[5b4aa4ead3089013...|[22.95,null]|[[1212883200000]]|       Putnam| [4]|           [0]|           THE FRONT|
|[5b4aa4ead3089013...|[24.95,null]|[[1212883200000]]|    Doubleday| [5]|           [0]|               SNUFF|
+--------------------+------------+-----------------+-------------+----+--------------+--------------------+
only showing top 5 

In [59]:
nyt2_2.show(10)

+--------------------+-----+--------------+-------------+----+--------------+--------------------+
|                  id|price|published_date|    publisher|rank|rank_last_week|               title|
+--------------------+-----+--------------+-------------+----+--------------+--------------------+
|5b4aa4ead30890135...| 27.0| 1212883200000|       Bantam|   1|             0|           ODD HOURS|
|5b4aa4ead30890135...|25.99| 1212883200000|Little, Brown|   2|             1|            THE HOST|
|5b4aa4ead30890135...|24.95| 1212883200000| St. Martin's|   3|             2|LOVE THE ONE YOU'...|
|5b4aa4ead30890135...|22.95| 1212883200000|       Putnam|   4|             0|           THE FRONT|
|5b4aa4ead30890135...|24.95| 1212883200000|    Doubleday|   5|             0|               SNUFF|
|5b4aa4ead30890135...|24.99| 1212883200000|Little, Brown|   6|             3|SUNDAYS AT TIFFANY’S|
|5b4aa4ead30890135...|26.95| 1212883200000|       Putnam|   7|             4|        PHANTOM PREY|
|5b4aa4ead

 # Task-4: Find the number of books whose current rank is better than the last week's rank.
  

In [60]:
nytDF2_2 = nytDF2_1.rdd\
                   .map(lambda x: (x['_id'][0],x['price'],x['published_date'][0][0],x['publisher'],int(x['rank'][0]),int(x['rank_last_week'][0])\
                                  ,x['title'])).toDF(['id','price','published_date','publisher','rank','rank_last_week','title'])

In [61]:
nytDF1_2 = nytDF1_1.rdd\
                   .map(lambda x:(x['_id'][0],x['amazon_product_url'],x['author'],x['bestsellers_date'][0][0],x['description'],int(x['weeks_on_list'][0]))).\
                    toDF(['id','amazon_product_url','author','bestsellers_date','description','weeks_on_list'])

In [62]:
nytDF1_2.show(5)

+--------------------+--------------------+-----------------+----------------+--------------------+-------------+
|                  id|  amazon_product_url|           author|bestsellers_date|         description|weeks_on_list|
+--------------------+--------------------+-----------------+----------------+--------------------+-------------+
|5b4aa4ead30890135...|http://www.amazon...|    Dean R Koontz|   1211587200000|Odd Thomas, who c...|            1|
|5b4aa4ead30890135...|http://www.amazon...|  Stephenie Meyer|   1211587200000|Aliens have taken...|            3|
|5b4aa4ead30890135...|http://www.amazon...|     Emily Giffin|   1211587200000|A woman's happy m...|            2|
|5b4aa4ead30890135...|http://www.amazon...|Patricia Cornwell|   1211587200000|A Massachusetts s...|            1|
|5b4aa4ead30890135...|http://www.amazon...|  Chuck Palahniuk|   1211587200000|An aging porn que...|            1|
+--------------------+--------------------+-----------------+----------------+----------

In [63]:
nyt2_2.show(5)

+--------------------+-----+--------------+-------------+----+--------------+--------------------+
|                  id|price|published_date|    publisher|rank|rank_last_week|               title|
+--------------------+-----+--------------+-------------+----+--------------+--------------------+
|5b4aa4ead30890135...| 27.0| 1212883200000|       Bantam|   1|             0|           ODD HOURS|
|5b4aa4ead30890135...|25.99| 1212883200000|Little, Brown|   2|             1|            THE HOST|
|5b4aa4ead30890135...|24.95| 1212883200000| St. Martin's|   3|             2|LOVE THE ONE YOU'...|
|5b4aa4ead30890135...|22.95| 1212883200000|       Putnam|   4|             0|           THE FRONT|
|5b4aa4ead30890135...|24.95| 1212883200000|    Doubleday|   5|             0|               SNUFF|
+--------------------+-----+--------------+-------------+----+--------------+--------------------+
only showing top 5 rows



In [64]:
nytDF1_1.createOrReplaceTempView('nyt1')
nyt2_2.createOrReplaceTempView('nyt2')

In [65]:
ss.sql('SELECT COUNT(DISTINCT id) FROM nyt2 WHERE rank < rank_last_week').show()

+------------------+
|count(DISTINCT id)|
+------------------+
|              1993|
+------------------+



 # Task-5: Print the number of books per number of weeks on the best seller's list.
Print it in descending order for the number of weeks.
E.g., 100 books with '10' weeks on the list, 200 books with '8' week on the list etc.
   

In [22]:
ss.sql('SELECT COUNT(id),weeks_on_list FROM nyt1 GROUP BY weeks_on_list ORDER BY weeks_on_list DESC ').show(10)

+---------+-------------+
|count(id)|weeks_on_list|
+---------+-------------+
|        1|          134|
|        1|          133|
|        1|          132|
|        1|          131|
|        1|          130|
|        1|          129|
|        1|          128|
|        1|          127|
|        1|          126|
|        1|          125|
+---------+-------------+
only showing top 10 rows



# Task-6 : Find the number of books with price less than 25 and that are exactly one week on the best seller's list.
   

In [67]:
comp = nytDF1_2.join(nyt2_2, nytDF1_2.id == nyt2_2.id,'INNER').select(nytDF1_2["id"],nytDF1_2["weeks_on_list"],nytDF1_2["author"],\
                                                                      nyt2_2["price"],nyt2_2["rank"],nyt2_2["rank_last_week"])

In [68]:
comp.columns

['id', 'weeks_on_list', 'author', 'price', 'rank', 'rank_last_week']

In [69]:
comp.filter('weeks_on_list= 1').filter('price < 25').count()

1177

 # Task-7: Does price of a book have an affect on the book being on the best seller's list?
   

In [70]:
comp.corr('price','weeks_on_list')

0.026038356119044665

In [27]:
comp.orderBy('price', ascending = False).show(10)

+--------------------+-------------+-----------+-----+----+--------------+
|                  id|weeks_on_list|     author|price|rank|rank_last_week|
+--------------------+-------------+-----------+-----+----+--------------+
|5b4aa4ead30890135...|            7|Ken Follett|   36|   8|            10|
|5b4aa4ead30890135...|            5|Ken Follett|   36|   7|             5|
|5b4aa4ead30890135...|            4|Ken Follett|   36|   5|             2|
|5b4aa4ead30890135...|           12|Ken Follett|   36|   8|             9|
|5b4aa4ead30890135...|            6|Ken Follett|   36|   8|             7|
|5b4aa4ead30890135...|            5|Ken Follett|   36|   7|             4|
|5b4aa4ead30890135...|            2|Ken Follett|   36|   2|             1|
|5b4aa4ead30890135...|           10|Ken Follett|   36|   7|             9|
|5b4aa4ead30890135...|            2|Ken Follett|   36|   2|             1|
|5b4aa4ead30890135...|           11|Ken Follett|   36|  13|            13|
+--------------------+---

In [46]:
comp.filter('price > 0').orderBy('price', ascending = True).show(10)

+--------------------+-------------+---------------+-----+----+--------------+
|                  id|weeks_on_list|         author|price|rank|rank_last_week|
+--------------------+-------------+---------------+-----+----+--------------+
|5b4aa4ead30890135...|            1|   Stephen King|14.99|   9|             0|
|5b4aa4ead30890135...|            3|   Stephen King|14.99|  16|            13|
|5b4aa4ead30890135...|            2|   Stephen King|14.99|  13|             9|
|5b4aa4ead30890135...|            3|Debbie Macomber|16.95|  14|            13|
|5b4aa4ead30890135...|            1|   Lisa Kleypas|16.95|   9|             0|
|5b4aa4ead30890135...|            1|Debbie Macomber|16.95|   2|             0|
|5b4aa4ead30890135...|            2|Debbie Macomber|16.95|   8|             2|
|5b4aa4ead30890135...|            1|Debbie Macomber|16.95|  14|             0|
|5b4aa4ead30890135...|            4|Debbie Macomber|16.95|  15|            10|
|5b4aa4ead30890135...|            1|Debbie Macomber|

In [32]:
comp.orderBy('weeks_on_list', ascending = False).filter('price>0').show(10)

+--------------------+-------------+----------------+-----+----+--------------+
|                  id|weeks_on_list|          author|price|rank|rank_last_week|
+--------------------+-------------+----------------+-----+----+--------------+
|5b4aa4ead30890135...|          108|Kathryn Stockett|24.95|  13|             9|
|5b4aa4ead30890135...|          107|Kathryn Stockett|24.95|   9|             8|
|5b4aa4ead30890135...|          106|Kathryn Stockett|24.95|   8|             4|
|5b4aa4ead30890135...|          105|Kathryn Stockett|24.95|   4|            11|
|5b4aa4ead30890135...|          104|Kathryn Stockett|24.95|  11|             0|
|5b4aa4ead30890135...|          103|Kathryn Stockett|24.95|  12|            14|
|5b4aa4ead30890135...|          102|Kathryn Stockett|24.95|  14|            12|
|5b4aa4ead30890135...|          101|Kathryn Stockett|24.95|  12|             8|
|5b4aa4ead30890135...|          100|Kathryn Stockett|24.95|   8|             4|
|5b4aa4ead30890135...|           99|Kath

In [47]:
comp.orderBy('weeks_on_list', ascending = True).filter('price>0').show(10)

+--------------------+-------------+--------------------+-----+----+--------------+
|                  id|weeks_on_list|              author|price|rank|rank_last_week|
+--------------------+-------------+--------------------+-----+----+--------------+
|5b4aa4ead30890135...|            1|       Kate Atkinson|27.99|   3|             0|
|5b4aa4ead30890135...|            1|           Brad Thor|27.99|   4|             0|
|5b4aa4ead30890135...|            1|Susan Elizabeth P...|25.99|   5|             0|
|5b4aa4ead30890135...|            1|       Karen Traviss|   20|  13|             0|
|5b4aa4ead30890135...|            1|James Patterson a...|27.99|   1|             0|
|5b4aa4ead30890135...|            1|Clive Cussler wit...|27.95|   4|             0|
|5b4aa4ead30890135...|            1|        Lev Grossman|26.95|   8|             0|
|5b4aa4ead30890135...|            1|    Michael Connelly|27.99|   1|             0|
|5b4aa4ead30890135...|            1|    Adriana Trigiani|25.99|   9|        

In [29]:
print 'There does not seem to be any relation of price and best seller week.'

There does not seem to be any relation of price and best seller week.


 # Task-8: How many unique authors are considered in the data?
 

In [192]:
ss.sql('SELECT COUNT(DISTINCT author) FROM nyt1').show()

+----------------------+
|count(DISTINCT author)|
+----------------------+
|                   738|
+----------------------+



  # Task-9: Is there any correlation between the authors and the price of their books?
  

In [329]:
comp.groupby('author').agg({'price': 'mean'}).show()

+--------------------+------------------+
|              author|        avg(price)|
+--------------------+------------------+
|    Elin Hilderbrand| 5.015344827586207|
|         Lisa Genova|               0.0|
|   Sharon Kay Penman|            14.475|
|          James Frey|            13.475|
|        Will Allison|               0.0|
|   Patricia Cornwell|16.002343750000005|
|          Tea Obreht|16.666666666666668|
|       Laurie R King|16.833333333333332|
|        Sarah Dunant|               0.0|
|        Tim Johnston|               0.0|
|Tom Clancy with P...|22.516666666666666|
|          Sara Gruen|               6.0|
|        Terry Brooks|15.736842105263158|
|        Lisa Wingate|               0.0|
|         Andre Dubus|               0.0|
|        Daniel Silva|11.030144927536236|
|Karen White, Beat...|               0.0|
|      Rachel Kushner|               0.0|
|      Jackie Collins|19.415714285714284|
|          Ann B Ross|               0.0|
+--------------------+------------

# Task-10: Are there any authors who are very well-known for being on the best seller's list?
    

In [204]:
print 'I interpret this question as "Show top authors based on the number of weeks their books were in the list"\n\n'
ss.sql('SELECT author,SUM(weeks_on_list) FROM nyt1 GROUP BY author ORDER BY SUM(weeks_on_list) DESC').show(10)

I interpret this question as "Show top authors based on the number of weeks their books were in the list"


+-----------------+------------------+
|           author|sum(weeks_on_list)|
+-----------------+------------------+
|    Anthony Doerr|              9045|
| Kathryn Stockett|              5886|
|George R R Martin|              3796|
|   Kristin Hannah|              3619|
|    Stieg Larsson|              3406|
|    Paula Hawkins|              3313|
|    Gillian Flynn|              3240|
|     John Grisham|              1994|
|  Stephenie Meyer|              1858|
|      Donna Tartt|              1830|
+-----------------+------------------+
only showing top 10 rows



# Task-11: Consider top-4 authors based on the number of books per author in the given data.
For these authors, find number of books per number of weeks on the best seller's list
only for the books for which the price is between $15 and $22.50.

In [223]:
ss.sql('SELECT COUNT(DISTINCT id),author FROM nyt1 GROUP BY author ORDER BY COUNT(id) DESC').show(4) 

+------------------+--------------+
|count(DISTINCT id)|        author|
+------------------+--------------+
|               226|  John Grisham|
|               222|David Baldacci|
|               174|Danielle Steel|
|               157|  Stephen King|
+------------------+--------------+
only showing top 4 rows



In [335]:
comp.groupby('author').agg({'id':'count'}).sort('count(id)',ascending=False).show()

+--------------------+---------+
|              author|count(id)|
+--------------------+---------+
|        John Grisham|      226|
|      David Baldacci|      222|
|      Danielle Steel|      174|
|        Stephen King|      157|
|       Anthony Doerr|      141|
|     Nicholas Sparks|      136|
|      Kristin Hannah|      128|
|       Stieg Larsson|      121|
|James Patterson a...|      119|
|   George R R Martin|      119|
|     James Patterson|      115|
|    Kathryn Stockett|      114|
|     Janet Evanovich|      111|
|        Stuart Woods|      107|
|       John Sandford|      107|
|    Michael Connelly|      104|
|       Paula Hawkins|       97|
|    Charlaine Harris|       94|
|           Lee Child|       94|
|        Jodi Picoult|       90|
+--------------------+---------+
only showing top 20 rows



# For these authors, find number of books per number of weeks on the best seller's list only for the books for which the price is between 15 and 22.50.

In [34]:
comp.groupby('author').agg({'id':'count'}).sort('count(id)',ascending=False).select('author').show(4)

+--------------+
|        author|
+--------------+
|  John Grisham|
|David Baldacci|
|Danielle Steel|
|  Stephen King|
+--------------+
only showing top 4 rows



In [35]:
top4 = ['John Grisham','David Baldacci','Danielle Steel','Stephen King']

In [36]:
comp.filter(comp.author.isin(top4)).filter('price >= 15').filter('price <= 22.50').sort('price').show()

+---+-------------+------+-----+----+--------------+
| id|weeks_on_list|author|price|rank|rank_last_week|
+---+-------------+------+-----+----+--------------+
+---+-------------+------+-----+----+--------------+



In [80]:
comp.filter(comp.author.isin(top4)).filter('price > 22.5').sort('price',ascending = True).show(2)

+--------------------+-------------+------------+-----+----+--------------+
|                  id|weeks_on_list|      author|price|rank|rank_last_week|
+--------------------+-------------+------------+-----+----+--------------+
|5b4aa4ead30890135...|            6|John Grisham| 24.0|   9|             7|
|5b4aa4ead30890135...|            7|John Grisham| 24.0|   7|             9|
+--------------------+-------------+------------+-----+----+--------------+
only showing top 2 rows



In [82]:
comp.filter(comp.author.isin(top4)).filter('price < 25').filter('price > 14').sort('price',ascending = True).show(10)

+--------------------+-------------+------------+-----+----+--------------+
|                  id|weeks_on_list|      author|price|rank|rank_last_week|
+--------------------+-------------+------------+-----+----+--------------+
|5b4aa4ead30890135...|            3|Stephen King|14.99|  16|            13|
|5b4aa4ead30890135...|            2|Stephen King|14.99|  13|             9|
|5b4aa4ead30890135...|            1|Stephen King|14.99|   9|             0|
|5b4aa4ead30890135...|            1|John Grisham| 24.0|   1|             0|
|5b4aa4ead30890135...|            3|John Grisham| 24.0|   4|             3|
|5b4aa4ead30890135...|           10|John Grisham| 24.0|  16|            13|
|5b4aa4ead30890135...|            4|John Grisham| 24.0|   6|             4|
|5b4aa4ead30890135...|            5|John Grisham| 24.0|   7|             6|
|5b4aa4ead30890135...|            7|John Grisham| 24.0|   7|             9|
|5b4aa4ead30890135...|            6|John Grisham| 24.0|   9|             7|
+-----------

In [357]:
sc.stop()